In [7]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import os
import pickle

# --- 데이터 로딩 ---
nadf = pd.DataFrame(pd.read_csv('./thermal_data_none.csv'))
frames_none = nadf.iloc[:,3:67]

hudf = pd.DataFrame(pd.read_csv('./thermal_data_human_one.csv'))
frames_hu = hudf.iloc[:,3:67]

# --- 온도 데이터를 0-255 범위로 스케일링하는 함수 ---
def scale_temperature_to_uint8(temp_data, min_val, max_val):
    scaled_data = (temp_data - min_val) / (max_val - min_val) * 255
    scaled_data = np.clip(scaled_data, 0, 255)
    return scaled_data.astype(np.uint8)

# --- 전역 온도 범위 계산 ---
global_min_temp = min(frames_none.min().min(), frames_hu.min().min())
global_max_temp = max(frames_none.max().max(), frames_hu.max().max())
print(f"온도 데이터 전체 범위: {global_min_temp:.2f}°C ~ {global_max_temp:.2f}°C")

# --- 배경 모델 클래스 ---
class ThermalBackgroundModel:
    def __init__(self, history=500, var_threshold=12, gaussian_ksize=(5, 5)):
        self.history = history
        self.var_threshold = var_threshold
        self.gaussian_ksize = gaussian_ksize
        self.gaussian_sigmaX = 0
        self.original_frame_width = 8
        self.original_frame_height = 8
        self.expanded_frame_width = 24
        self.expanded_frame_height = 24
        
        # MOG2 배경 제거기 생성
        self.fgbg = cv2.createBackgroundSubtractorMOG2(
            history=self.history, 
            varThreshold=self.var_threshold, 
            detectShadows=False
        )
        
        self.is_trained = False
        self.training_data = []
        
    def train_background_model(self, background_frames, global_min_temp, global_max_temp):
        """배경 프레임들로 모델 학습"""
        print("--- 배경 모델 학습 시작 ---")
        print(f"학습 프레임 수: {len(background_frames)}")
        
        for i in range(len(background_frames.index)):
            temp_frame_2d = background_frames.iloc[i,:].values.reshape(
                (self.original_frame_height, self.original_frame_width)
            )
            scaled_frame = scale_temperature_to_uint8(temp_frame_2d, global_min_temp, global_max_temp)
            
            # 24x24로 확대
            expanded_frame = cv2.resize(scaled_frame, 
                                      (self.expanded_frame_width, self.expanded_frame_height), 
                                      interpolation=cv2.INTER_NEAREST)
            
            # 가우시안 블러 적용
            blurred_frame = cv2.GaussianBlur(expanded_frame, self.gaussian_ksize, self.gaussian_sigmaX)
            
            # MOG2 학습
            _ = self.fgbg.apply(blurred_frame)
            
            # 학습 데이터 저장 (원본 온도값으로)
            self.training_data.append(temp_frame_2d.flatten())
            
            if (i + 1) % 500 == 0:
                print(f"  {i + 1}/{len(background_frames)} 프레임 학습 완료...")
        
        self.is_trained = True
        print("--- 배경 모델 학습 완료 ---")
        
    def process_frame(self, frame_64pixels, global_min_temp, global_max_temp):
        """64픽셀 데이터를 처리하여 열원 추출"""
        if not self.is_trained:
            raise ValueError("모델이 학습되지 않았습니다. train_background_model()을 먼저 실행하세요.")
        
        # 8x8로 reshape
        temp_frame_2d = frame_64pixels.reshape((self.original_frame_height, self.original_frame_width))
        scaled_frame = scale_temperature_to_uint8(temp_frame_2d, global_min_temp, global_max_temp)
        
        # 24x24로 확대
        expanded_frame = cv2.resize(scaled_frame, 
                                  (self.expanded_frame_width, self.expanded_frame_height), 
                                  interpolation=cv2.INTER_NEAREST)
        
        # 가우시안 블러 적용
        blurred_frame = cv2.GaussianBlur(expanded_frame, self.gaussian_ksize, self.gaussian_sigmaX)
        
        # 전경 감지
        fg_mask = self.fgbg.apply(blurred_frame)
        
        # 열원만 추출
        heat_source_only = cv2.bitwise_and(blurred_frame, blurred_frame, mask=fg_mask)
        
        return {
            'original_expanded': expanded_frame,
            'blurred_expanded': blurred_frame,
            'foreground_mask': fg_mask,
            'heat_source_only': heat_source_only,
            'original_temp_2d': temp_frame_2d
        }
    
    def save_model(self, filepath):
        """모델 저장"""
        model_data = {
            'fgbg_state': None,  # MOG2는 직접 저장 불가
            'history': self.history,
            'var_threshold': self.var_threshold,
            'gaussian_ksize': self.gaussian_ksize,
            'training_data': self.training_data,
            'is_trained': self.is_trained,
            'model_params': {
                'original_frame_width': self.original_frame_width,
                'original_frame_height': self.original_frame_height,
                'expanded_frame_width': self.expanded_frame_width,
                'expanded_frame_height': self.expanded_frame_height
            }
        }
        
        with open(filepath, 'wb') as f:
            pickle.dump(model_data, f)
        print(f"배경 모델이 저장되었습니다: {filepath}")

# --- 열원 판별 함수 (64픽셀 입력) ---
def detect_heat_source_from_64pixels(frame_64pixels, background_model, global_min_temp, global_max_temp, 
                                    min_area_threshold=10, min_intensity_threshold=50):
    """
    64개 픽셀 데이터로부터 열원을 감지하는 함수
    
    Args:
        frame_64pixels: 64개 픽셀의 온도 데이터 (1D array)
        background_model: 학습된 ThermalBackgroundModel 인스턴스
        global_min_temp: 전역 최소 온도
        global_max_temp: 전역 최대 온도
        min_area_threshold: 최소 영역 크기
        min_intensity_threshold: 최소 강도 임계값
    
    Returns:
        dict: 열원 감지 결과
    """
    # 프레임 처리
    processed = background_model.process_frame(frame_64pixels, global_min_temp, global_max_temp)
    
    fg_mask = processed['foreground_mask']
    heat_source_only = processed['heat_source_only']
    
    # 전경 픽셀 수 계산
    foreground_pixels = np.sum(fg_mask > 0)
    total_pixels = fg_mask.size
    foreground_percentage = (foreground_pixels / total_pixels) * 100
    
    # 열원 영역의 평균 강도
    heat_intensity = 0
    if foreground_pixels > 0:
        heat_intensity = np.mean(heat_source_only[fg_mask > 0])
    
    # 연결된 구성 요소 분석
    num_labels, labels = cv2.connectedComponents(fg_mask)
    significant_components = 0
    
    for i in range(1, num_labels):
        component_size = np.sum(labels == i)
        if component_size >= min_area_threshold:
            significant_components += 1
    
    # 열원 존재 여부 판단
    heat_source_detected = (
        foreground_pixels >= min_area_threshold and 
        heat_intensity >= min_intensity_threshold and
        significant_components > 0
    )
    
    # 열원 강도 분류
    if heat_intensity >= 200:
        intensity_level = "매우 높음"
    elif heat_intensity >= 150:
        intensity_level = "높음"
    elif heat_intensity >= 100:
        intensity_level = "보통"
    elif heat_intensity >= 50:
        intensity_level = "낮음"
    else:
        intensity_level = "매우 낮음"
    
    # 온도 통계 계산
    original_temp = processed['original_temp_2d']
    current_max_temp = original_temp.max()
    current_avg_temp = original_temp.mean()
    current_min_temp = original_temp.min()
    
    return {
        'detected': heat_source_detected,
        'confidence': min(100, (heat_intensity / 255) * 50 + (foreground_percentage * 2)),
        'intensity_level': intensity_level,
        'heat_intensity': heat_intensity,
        'foreground_pixels': foreground_pixels,
        'foreground_percentage': foreground_percentage,
        'num_components': significant_components,
        'temperature_stats': {
            'max_temp': current_max_temp,
            'avg_temp': current_avg_temp,
            'min_temp': current_min_temp
        },
        'processed_data': processed
    }

# --- 데이터 저장 함수 ---
def save_background_model_and_filtered_data(background_model, processed_results, 
                                          global_min_temp, global_max_temp, 
                                          output_dir="thermal_model_output"):
    """배경 모델과 필터링된 데이터를 저장"""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # 1. 배경 모델 저장
    model_file = os.path.join(output_dir, f"thermal_background_model_{timestamp}.pkl")
    background_model.save_model(model_file)
    
    # 2. Heat Source Only 데이터 저장 (원본 64픽셀 형태로)
    heat_source_data = []
    detection_results = []
    
    for i, result in enumerate(processed_results):
        # Heat source only를 다시 8x8로 축소하여 64픽셀 데이터로 변환
        heat_source_24x24 = result['processed_data']['heat_source_only']
        heat_source_8x8 = cv2.resize(heat_source_24x24, (8, 8), interpolation=cv2.INTER_AREA)
        
        # 온도값으로 역변환
        heat_source_temp = global_min_temp + (heat_source_8x8.flatten() / 255) * (global_max_temp - global_min_temp)
        
        # 64픽셀 데이터로 저장
        row_data = {'frame_id': i}
        for j in range(64):
            row_data[f'pixel_{j}'] = heat_source_temp[j]
        heat_source_data.append(row_data)
        
        # 감지 결과 저장
        detection_results.append({
            'frame_id': i,
            'detected': result['detected'],
            'confidence': result['confidence'],
            'intensity_level': result['intensity_level'],
            'heat_intensity': result['heat_intensity'],
            'foreground_pixels': result['foreground_pixels'],
            'foreground_percentage': result['foreground_percentage'],
            'num_components': result['num_components'],
            'max_temp': result['temperature_stats']['max_temp'],
            'avg_temp': result['temperature_stats']['avg_temp'],
            'min_temp': result['temperature_stats']['min_temp']
        })
    
    # Heat Source Only 데이터 CSV 저장
    heat_source_df = pd.DataFrame(heat_source_data)
    heat_source_file = os.path.join(output_dir, f"heat_source_only_64pixels_{timestamp}.csv")
    heat_source_df.to_csv(heat_source_file, index=False)
    
    # 감지 결과 CSV 저장
    detection_df = pd.DataFrame(detection_results)
    detection_file = os.path.join(output_dir, f"heat_detection_results_{timestamp}.csv")
    detection_df.to_csv(detection_file, index=False)
    
    # 3. 배경 모델 학습 데이터 저장 (원본 none 데이터)
    training_data_rows = []
    for i, training_frame in enumerate(background_model.training_data):
        row_data = {'frame_id': i}
        for j in range(64):
            row_data[f'pixel_{j}'] = training_frame[j]
        training_data_rows.append(row_data)
    
    training_df = pd.DataFrame(training_data_rows)
    training_file = os.path.join(output_dir, f"background_training_data_{timestamp}.csv")
    training_df.to_csv(training_file, index=False)
    
    # 통계 출력
    total_frames = len(processed_results)
    detected_frames = sum(1 for result in processed_results if result['detected'])
    detection_rate = (detected_frames / total_frames) * 100
    
    print(f"\n=== 저장 완료 ===")
    print(f"배경 모델: {model_file}")
    print(f"Heat Source Only 데이터: {heat_source_file}")
    print(f"감지 결과: {detection_file}")
    print(f"배경 학습 데이터: {training_file}")
    print(f"\n=== 처리 통계 ===")
    print(f"총 프레임 수: {total_frames}")
    print(f"열원 감지된 프레임: {detected_frames}")
    print(f"열원 감지율: {detection_rate:.1f}%")
    
    # 강도별 분포
    intensity_counts = {}
    for result in processed_results:
        level = result['intensity_level']
        intensity_counts[level] = intensity_counts.get(level, 0) + 1
    
    print(f"\n=== 열원 강도별 분포 ===")
    for level, count in intensity_counts.items():
        percentage = (count / total_frames) * 100
        print(f"{level}: {count}프레임 ({percentage:.1f}%)")
    
    return {
        'model_file': model_file,
        'heat_source_file': heat_source_file,
        'detection_file': detection_file,
        'training_file': training_file,
        'stats': {
            'total_frames': total_frames,
            'detected_frames': detected_frames,
            'detection_rate': detection_rate,
            'intensity_distribution': intensity_counts
        }
    }

# --- 시각화 함수 ---
def visualize_detection_result(detection_result, frame_idx, global_min_temp, global_max_temp):
    """감지 결과 시각화"""
    processed = detection_result['processed_data']
    
    # fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    # fig.suptitle(f'열원 감지 결과 - Frame {frame_idx}\n'
    #             f'감지: {"🔥 예" if detection_result["detected"] else "❄️ 아니오"} | '
    #             f'신뢰도: {detection_result["confidence"]:.1f}% | '
    #             f'강도: {detection_result["intensity_level"]}', 
    #             fontsize=14, fontweight='bold',
    #             color='red' if detection_result['detected'] else 'blue')
    
    # # 원본 8x8 온도 데이터
    # axes[0,0].imshow(processed['original_temp_2d'], cmap='hot')
    # axes[0,0].set_title(f'원본 온도 데이터 (8x8)\nMax: {detection_result["temperature_stats"]["max_temp"]:.1f}°C')
    # axes[0,0].axis('off')
    
    # # 확대된 24x24 데이터
    # axes[0,1].imshow(processed['original_expanded'], cmap='hot')
    # axes[0,1].set_title('확대된 데이터 (24x24)')
    # axes[0,1].axis('off')
    
    # # 전경 마스크
    # axes[1,0].imshow(processed['foreground_mask'], cmap='gray')
    # axes[1,0].set_title(f'전경 마스크\n감지된 픽셀: {detection_result["foreground_pixels"]}개')
    # axes[1,0].axis('off')
    
    # # Heat Source Only
    # axes[1,1].imshow(processed['heat_source_only'], cmap='hot')
    # axes[1,1].set_title(f'열원만 추출\n구성요소: {detection_result["num_components"]}개')
    # axes[1,1].axis('off')
    
    # plt.tight_layout()
    
    # return fig

# --- 메인 실행 코드 ---
def main():
    print("=== 열원 감지 시스템 시작 ===")
    
    # 1. 배경 모델 생성 및 학습
    background_model = ThermalBackgroundModel(history=len(frames_none), var_threshold=16)
    background_model.train_background_model(frames_none, global_min_temp, global_max_temp)
    
    # 2. 열원 포함 프레임들 처리
    print("\n--- 열원 포함 프레임 처리 시작 ---")
    processed_results = []
    
    for i in range(len(frames_hu.index)):
        frame_64pixels = frames_hu.iloc[i,:].values
        
        # 열원 감지 수행
        detection_result = detect_heat_source_from_64pixels(
            frame_64pixels, background_model, global_min_temp, global_max_temp
        )
        
        processed_results.append(detection_result)
        
        # 100프레임마다 또는 열원이 감지되면 출력
        if (i + 1) % 100 == 0 or detection_result['detected']:
            status = "🔥 감지됨" if detection_result['detected'] else "❄️ 없음"
            print(f"프레임 {i:4d}: {status} | "
                  f"신뢰도: {detection_result['confidence']:5.1f}% | "
                  f"강도: {detection_result['intensity_level']:8s} | "
                  f"최고온도: {detection_result['temperature_stats']['max_temp']:5.1f}°C")
    
    print("--- 열원 포함 프레임 처리 완료 ---")
    
    # 3. 모델과 데이터 저장
    save_results = save_background_model_and_filtered_data(
        background_model, processed_results, global_min_temp, global_max_temp
    )
    
    # 4. 결과 시각화 (몇 개 프레임만)
    print("\n--- 결과 시각화 ---")
    display_indices = [0, min(100, len(processed_results)-1), min(500, len(processed_results)-1)]
    
    # for idx in display_indices:
    #     if idx < len(processed_results):
    #         result = processed_results[idx]
    #         print(f"\n프레임 {idx} 상세 결과:")
    #         print(f"  열원 감지: {'예' if result['detected'] else '아니오'}")
    #         print(f"  신뢰도: {result['confidence']:.1f}%")
    #         print(f"  강도 레벨: {result['intensity_level']}")
    #         print(f"  최고온도: {result['temperature_stats']['max_temp']:.1f}°C")
            
    #         fig = visualize_detection_result(result, idx, global_min_temp, global_max_temp)
    #         plt.show()
    
    return background_model, processed_results, save_results

# --- 64픽셀 데이터로 열원 감지하는 간단한 함수 (사용 예시) ---
def simple_heat_detection(frame_64pixels, model_path, global_min_temp, global_max_temp):
    """
    저장된 모델로 64픽셀 데이터의 열원을 감지하는 간단한 함수
    
    Args:
        frame_64pixels: 64개 픽셀의 온도 데이터
        model_path: 저장된 모델 파일 경로 (실제로는 새로 학습된 모델 사용)
        global_min_temp: 전역 최소 온도
        global_max_temp: 전역 최대 온도
    
    Returns:
        dict: 감지 결과
    """
    # 실제 사용시에는 저장된 모델을 로드해야 하지만,
    # 여기서는 현재 세션의 모델을 사용하는 예시
    print("⚠️ 실제 사용시에는 train_background_model()을 먼저 실행하세요.")
    
    # 예시 결과 (실제로는 모델을 로드하여 사용)
    return {
        'detected': False,
        'confidence': 0.0,
        'message': '모델이 로드되지 않았습니다. main() 함수를 먼저 실행하세요.'
    }

if __name__ == "__main__":
    # 실행
    background_model, processed_results, save_results = main()
    
    print(f"\n=== 최종 결과 ===")
    print(f"열원 감지율: {save_results['stats']['detection_rate']:.1f}%")
    print(f"총 처리 프레임: {save_results['stats']['total_frames']}개")
    print(f"저장된 파일들:")
    for key, filepath in save_results.items():
        if key != 'stats':
            print(f"  - {key}: {filepath}")

온도 데이터 전체 범위: 20.50°C ~ 29.00°C
=== 열원 감지 시스템 시작 ===
--- 배경 모델 학습 시작 ---
학습 프레임 수: 4073
  500/4073 프레임 학습 완료...
  1000/4073 프레임 학습 완료...
  1500/4073 프레임 학습 완료...
  2000/4073 프레임 학습 완료...
  2500/4073 프레임 학습 완료...
  3000/4073 프레임 학습 완료...
  3500/4073 프레임 학습 완료...
  4000/4073 프레임 학습 완료...
--- 배경 모델 학습 완료 ---

--- 열원 포함 프레임 처리 시작 ---
프레임   12: 🔥 감지됨 | 신뢰도:  37.3% | 강도: 높음       | 최고온도:  27.0°C
프레임   30: 🔥 감지됨 | 신뢰도:  25.8% | 강도: 낮음       | 최고온도:  26.0°C
프레임   32: 🔥 감지됨 | 신뢰도:  20.9% | 강도: 낮음       | 최고온도:  25.5°C
프레임   35: 🔥 감지됨 | 신뢰도:  18.8% | 강도: 낮음       | 최고온도:  25.8°C
프레임   66: 🔥 감지됨 | 신뢰도:  23.7% | 강도: 낮음       | 최고온도:  26.2°C
프레임   71: 🔥 감지됨 | 신뢰도:  32.4% | 강도: 낮음       | 최고온도:  25.0°C
프레임   76: 🔥 감지됨 | 신뢰도:  28.6% | 강도: 낮음       | 최고온도:  25.2°C
프레임   86: 🔥 감지됨 | 신뢰도:  20.2% | 강도: 낮음       | 최고온도:  25.8°C
프레임   87: 🔥 감지됨 | 신뢰도:  31.7% | 강도: 보통       | 최고온도:  26.8°C
프레임   88: 🔥 감지됨 | 신뢰도:  33.3% | 강도: 낮음       | 최고온도:  26.5°C
프레임   92: 🔥 감지됨 | 신뢰도:  24.0% | 강도: 낮음       | 최고온도:  25.8